In [1]:
# Version 1.1
# Alyssa Bulatek
# April 2020

# Import necessary packages.
import numpy as np
import pandas as pd
from scipy.integrate import simps
from mpmath import besselj, besseljzero
import os

##############
# Note that this code does not work directly with the FF files exported by CST.
# I have to remove the line of dashes as well as the spaces between words 
# within a single heading in the .txt files manually. Some future work for
# this script would involve automating this editing as part of the script, as well as
# scripting this to run on each of the files in a directory so the filenames 
# do not need to be entered manually. I'm not an expert coder, though, so I
# haven't done this yet! 
##############

# CHANGE USER-SPECIFIED PARAMETERS HERE.

modeType = "TM" # Either TE or TM as a string.
m, n = 1, 3 # Set values for the desired mode.
path = "/Users/alyssabulatek/Desktop/Mode_calculations/Final_Tef/TM13_smWindow" # Path to directory.
filename = "4200FF_E_14p88_p1and3.txt" # Name of farfield file as a string.
a = 0.45 # Aperture radius in m.

#########################################

freq_string = filename[:4] # Frequency as a string for in-file output.
freq = (int(freq_string)/1000)*(10**9) # Frequency of the observation in Hz.

# Extract radiation pattern from file.
filepath = path+"/"+filename
df = pd.read_csv(filepath,delim_whitespace=True)
phi_deg = df['Phi[deg.]'].values
phi_deg = df['Phi[deg.]'].values
phi_rad = (phi_deg*np.pi)/180.

if modeType == "TE":
    E_phi = df['Abs(Phi)[V/m]'].values
    Phase_phi = df['Phase(Phi)[deg.]'].values
    Phase_phi_rad = (Phase_phi*np.pi)/180.
    G_phi = E_phi*np.cos(Phase_phi_rad) + 1j*E_phi*np.sin(Phase_phi_rad)
elif modeType == "TM":
    E_theta = df['Abs(Theta)[V/m]'].values
    Phase_theta = df['Phase(Theta)[deg.]'].values
    Phase_theta_rad = (Phase_theta*np.pi)/180.
    G_theta = E_theta*np.cos(Phase_theta_rad) + 1j*E_theta*np.sin(Phase_theta_rad)
else:
    print("Input error: mode type (TE or TM) not specified.")
    
# Calculate general quantities.
wavelength = (2.998*(10**8))/freq # Frequency of the observation in meters.
k = (2*np.pi)/wavelength # Wavenumber of the observation in inverse meters.

# Integrate over the radiation pattern.
if modeType == "TE":
    integrand_A = G_phi*np.sin(m*phi_rad)
    A_pq_int = simps(integrand_A, phi_rad)
elif modeType == "TM":
    integrand_B = G_theta*np.cos(m*phi_rad)
    B_pq_int = simps(integrand_B, phi_rad)
else:
    print("Input error: mode type (TE or TM) not specified.")
    
# Calculate uppercase and lowercase coefficients.
if modeType == "TE": 
    # Calculate A_pq.
    chi_pq_prime = besseljzero(m,n,derivative=1) # order, nth zero, derivative
    J_p_double_prime = besselj(m,chi_pq_prime,derivative=2) # order, location, derivative
    A_pq = (2/(1+np.sqrt(1-(chi_pq_prime/(k*a))**2)))*((2*chi_pq_prime)/(np.pi*J_p_double_prime))*A_pq_int
    # Calculate a_mn.
    J_m = besselj(m,chi_pq_prime,derivative=0)
    a_mn = -A_pq/((1j**m)*k*J_m*(chi_pq_prime**2)*(a**2))
elif modeType == "TM":
    # Calculate B_pq. 
    chi_pq = besseljzero(m,n,derivative=0) # order, nth zero, derivative
    J_p_plus_one = besselj((m+1),chi_pq,derivative=0) # order, location, derivative
    B_pq = (2/(1+np.sqrt(1-(chi_pq/(k*a))**2)))*(2/(np.pi*J_p_plus_one))*B_pq_int
    # Calculate b_mn.
    J_m_prime = besselj(m,chi_pq,derivative=1)
    b_mn = B_pq/((1j**m)*k*J_m_prime*chi_pq*(a**2))
else:
    print("Input error: mode type (TE or TM) not specified.")
    
# Print out magnitudes of coefficients.
if modeType == "TE":
    print("The coefficient of the TE",m,n,"mode at",freq_string,"MHz is")
    print(abs(a_mn))
elif modeType == "TM":
    print("The coefficient of the TM",m,n,"mode at",freq_string,"MHz is")
    print(abs(b_mn))
else:
    print("Input error: mode type (TE or TM) not specified.")

The coefficient of the TM 1 3 mode at 4200 MHz is
5.3974226316313
